In [7]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np

def process_frame():
    ret, frame = cap.read()
    if not ret:
        print("Error al leer el fotograma.")
        return

    # Aplicar la segmentación HSV
    segmented_frame = segmentar(frame, hsv_upper_h.get(), hsv_upper_s.get(), hsv_upper_v.get())

    update_canvas(frame, canvas_frame, 0)
    update_canvas(segmented_frame, canvas_segmented, 1)

    ventana.after(20, process_frame)

def update_canvas(image, canvas, position):
    try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = resize_image(image, 300)
        image = Image.fromarray(image)
        image_tk = ImageTk.PhotoImage(image=image)
        canvas.delete(f"image_{position}")  # Eliminar la imagen anterior si existe
        canvas.create_image(0, 0, anchor=tk.NW, image=image_tk, tags=f"image_{position}")
        canvas.image = image_tk
    except Exception as e:
        print(f"Error en update_canvas: {e}")

def resize_image(image, max_width):
    original_height, original_width, _ = image.shape
    ratio = max_width / original_width
    height = int(original_height * ratio)
    return cv2.resize(image, (max_width, height))

def segmentar(image, h_upper, s_upper, v_upper):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    upper_np = np.array([h_upper, s_upper, v_upper])
    mask = cv2.inRange(hsv, np.array([0, 0, 0]), upper_np)
    segmented = cv2.bitwise_and(image, image, mask=mask)
    return segmented

def aplicar_segmentacion():
    # Obtener los valores HSV Upper ingresados por el usuario
    h_upper = hsv_upper_h.get()
    s_upper = hsv_upper_s.get()
    v_upper = hsv_upper_v.get()

    # Actualizar la etiqueta con los valores HSV Upper ingresados
    label_hsv_upper.config(text=f"HSV Upper: [{h_upper}, {s_upper}, {v_upper}]")

    # Llamar a la función de segmentación con los nuevos valores HSV Upper
    ret, frame = cap.read()
    if ret:
        segmented_frame = segmentar(frame, h_upper, s_upper, v_upper)
        update_canvas(segmented_frame, canvas_segmented, 1)

# Crear la interfaz gráfica
ventana = tk.Tk()
ventana.title("Segmentación HSV en Tiempo Real con Tkinter")

cap = cv2.VideoCapture(0)

# Canvas para mostrar las imágenes
canvas_frame = tk.Canvas(ventana, width=300, height=225)
canvas_frame.grid(row=0, column=0, padx=10, pady=10)

canvas_segmented = tk.Canvas(ventana, width=300, height=225)
canvas_segmented.grid(row=0, column=1, padx=10, pady=10)

# Controles deslizantes para valores HSV Upper (en horizontal)
hsv_upper_h = tk.Scale(ventana, label="Hue (H) Upper", from_=0, to=179, orient=tk.HORIZONTAL)
hsv_upper_h.set(60)
hsv_upper_h.grid(row=1, column=1, padx=10, pady=10)

hsv_upper_s = tk.Scale(ventana, label="Saturation (S) Upper", from_=0, to=255, orient=tk.HORIZONTAL)
hsv_upper_s.set(220)
hsv_upper_s.grid(row=2, column=1, padx=10, pady=10)

hsv_upper_v = tk.Scale(ventana, label="Value (V) Upper", from_=0, to=255, orient=tk.HORIZONTAL)
hsv_upper_v.set(180)
hsv_upper_v.grid(row=3, column=1, padx=10, pady=10)

# Etiqueta para mostrar los valores HSV Upper
label_hsv_upper = tk.Label(ventana, text="HSV Upper: [60, 220, 180]")
label_hsv_upper.grid(row=4, column=1, padx=10, pady=5)

# Botón para aplicar la segmentación
button_apply_segmentation = tk.Button(ventana, text="Aplicar Segmentación", command=aplicar_segmentacion)
button_apply_segmentation.grid(row=5, column=1, padx=10, pady=10)

# Agregar función para manejar el cierre de la ventana
def on_closing():
    cap.release()
    ventana.destroy()

ventana.protocol("WM_DELETE_WINDOW", on_closing)

# Llamar a la función process_frame inicialmente
ventana.after(20, process_frame)

ventana.mainloop()